## This notebook calculates the following Climate Domain Heat Loss metrics:
* average number of age adjusted heat related hospitalizations per 10,000

In [22]:
import os
import sys
import pandas as pd
import io
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
) 
pd.set_option('display.max_columns', None)

In [23]:
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/climate_risk/extreme_heat/loss/public_health_institute/emergency_department_visits/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'heat_related_illness - emergency department visits.csv'


In [24]:
heat_dept_visits_data = pd.read_csv('heat_related_illness - emergency department visits.csv')

In [25]:
# rename columns, eliminate statewide data, and make all strings lowercase
heat_dept_visits_cleaned = heat_dept_visits_data.drop(columns={'Lower\n95% Limit', 'Upper\n95% Limit', 'Total'})
heat_dept_visits_cleaned = heat_dept_visits_cleaned.rename(columns={'Age-adjusted\nrate per 100,000': 'age_adjust_per_100000'})
heat_dept_visits_cleaned = heat_dept_visits_cleaned[heat_dept_visits_cleaned['County'] != 'California/Statewide']
heat_dept_visits_cleaned = heat_dept_visits_cleaned.applymap(lambda s: s.lower() if type(s) == str else s)
heat_dept_visits_cleaned.columns = heat_dept_visits_cleaned.columns.str.lower()

C:\Users\jespi\AppData\Local\Temp\ipykernel_33192\1805106689.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  heat_dept_visits_cleaned = heat_dept_visits_cleaned.applymap(lambda s: s.lower() if type(s) == str else s)


In [26]:
heat_dept_visits_cleaned

,county,age_adjust_per_100000,year
1,alameda,4.1,2005
2,alpine,NaN,2005
3,amador,NaN,2005
4,butte,21.1,2005
5,calaveras,34.7,2005
...,...,...,...
939,tulare,20.3,2020
940,tuolumne,NaN,2020
941,ventura,10.5,2020
942,yolo,22.7,2020


In [27]:
# checking results below with Alameda county
alameda = heat_dept_visits_cleaned[heat_dept_visits_cleaned['county'] == 'alameda']
alameda

,county,age_adjust_per_100000,year
1,alameda,4.1,2005
60,alameda,13.5,2006
119,alameda,5.8,2007
178,alameda,9.6,2008
237,alameda,7.4,2009
296,alameda,5.6,2010
355,alameda,5.2,2011
414,alameda,5.7,2012
473,alameda,7.0,2013
532,alameda,5.5,2014


In [28]:
# group the data by county and find the average of age adjusted heat hospitalizations column
heat_dept_visits_grouped = heat_dept_visits_cleaned.groupby(['county']).agg({
    'age_adjust_per_100000' : 'mean'
}).reset_index()

heat_dept_visits_grouped = heat_dept_visits_grouped.rename(columns={'age_adjust_per_100000': 'avg_age_adjust_per_100000'})

# multiply results by 10 to go from per 100,000 to 10,000
heat_dept_visits_grouped['avg_age_adjust_heat_hospitalizations_per_10000'] = heat_dept_visits_grouped['avg_age_adjust_per_100000'] * 10

# drop unnecessary columns
heat_dept_visits_grouped = heat_dept_visits_grouped.drop(columns={'avg_age_adjust_per_100000'})

print(len(heat_dept_visits_grouped))
heat_dept_visits_grouped.head(15)

58


,county,avg_age_adjust_heat_hospitalizations_per_10000
0,alameda,74.250000
1,alpine,NaN
2,amador,423.888889
3,butte,271.437500
4,calaveras,399.111111
5,colusa,768.000000
6,contra costa,113.562500
7,del norte,NaN
8,el dorado,180.500000
9,fresno,230.812500


In [29]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

ca_tract_county

C:\Users\jespi\AppData\Local\Temp\ipykernel_33192\3785741718.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)


,tract,countyfp,county
0,06085504321,085,santa clara
1,06085504410,085,santa clara
2,06085507003,085,santa clara
3,06085507004,085,santa clara
4,06085502204,085,santa clara
...,...,...,...
9124,06059001303,059,orange
9125,06059001304,059,orange
9126,06059001401,059,orange
9127,06013367200,013,contra costa


In [30]:
# merge data with CA county/tracts
heat_dept_visits_merge = pd.merge(ca_tract_county, heat_dept_visits_grouped, how='left', on='county')
heat_dept_visits_merge

,tract,countyfp,county,avg_age_adjust_heat_hospitalizations_per_10000
0,06085504321,085,santa clara,61.8125
1,06085504410,085,santa clara,61.8125
2,06085507003,085,santa clara,61.8125
3,06085507004,085,santa clara,61.8125
4,06085502204,085,santa clara,61.8125
...,...,...,...,...
9124,06059001303,059,orange,63.8125
9125,06059001304,059,orange,63.8125
9126,06059001401,059,orange,63.8125
9127,06013367200,013,contra costa,113.5625


In [31]:
heat_dept_visits_merge.to_csv('climate_heat_hospitalizations_metric.csv', index=False)

## Function Call

In [34]:
@append_metadata
def heat_hospitalizations_upload(input_csv, export=False, varname=''):
    '''
    Uploads the heat loss metric to the S3 bucket. The metric is:
    
    * average number of age adjusted heat related hospitalizations per 10,000
    
    Data for this metric was sourced Public Health Institute - Tracking California:
    https://www.phi.org/our-work/programs/tracking-california/

    Methods
    -------
    Relevant columns were isolated and renamed.
    California total rows were ommitted.
    Data was grouped by county and had its heat hospitalizations averaged.
    Values were multipled by 10 to get our per 10,000 portion of the metric.
    Data was merged to California census tract data.
    
    Parameters
    ----------
    input_csv: string
        csv heat loss data 
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI heat loss metrics to AWS
        True = will upload resulting df containing CAL CRAI heat loss metrics to AWS

    Script
    ------
    climate_heat_loss_hospitalizations.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: relevant columns were isolated and renamed')
    print('Data transformation: data was grouped by county and averaged.')
    print('Data transformation: data was merged with California census tracts.') 
 
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [input_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{input_csv} uploaded to AWS.')
 
    #if os.path.exists(input_csv):
    #   os.remove(input_csv)

In [35]:
input_csv = 'climate_heat_hospitalizations_metric.csv'

var = 'climate_phi_heat_emergency_dept_visits'

# Process the data and export
heat_hospitalizations_upload(input_csv, export=True, varname='test') #var
